In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.display import clear_output

In [2]:
import os
import sys

sys.path.append('..')

from src.data.boardgame_resource import BoardgameResource

In [ ]:
from bs4 import BeautifulSoup
import re
import requests

sitemaps = requests.get('http://www.boardgamegeek.com/sitemapindex').content
sitemaps = BeautifulSoup(sitemaps, 'lxml').select('sitemapindex > sitemap > loc')
sitemaps = [str(s.get_text()) for s in sitemaps]
sitemaps = [s.replace('\n', '') for s in sitemaps]
sitemaps = [s.replace('\t', '') for s in sitemaps]
sitemaps = [s for s in sitemaps if 'geekitems_boardgame_' in s]

In [ ]:
boardgame_urls = []
for sitemap in sitemaps:
    sitemap = requests.get(sitemap).content
    urls = BeautifulSoup(sitemap, 'lxml').select('urlset > url > loc')
    urls = [str(u.get_text()) for u in urls]
    urls = [u.replace('\n', '') for u in urls]
    urls = [u.replace('\t', '') for u in urls]
    boardgame_urls.extend(urls)

In [ ]:
boardgame_ids = [int(re.compile('[0-9]+').search(u).group(0)) for u in boardgame_urls]

In [36]:
import pickle

pickle.dump(boardgame_ids, open('bgids.p', 'w+'))

In [3]:
import pickle

boardgame_ids = pickle.load(open('bgids.p', 'rb'))
len(boardgame_ids)

86855

In [4]:
from src.data.boardgame_crawler import BoardgameCrawler
save_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw')

In [ ]:
for i, id in enumerate(boardgame_ids):
    
    clear_output()
    print(i, id)
    
    bg = BoardgameCrawler({'id': id})
    bg.crawl(save_dir=save_dir)

(15735, 17108)


In [ ]:
time.time()

In [15]:
'%.2f' % (float(3)/50*100) + '%'

'6.00%'

In [ ]:
test

In [ ]:
'&#10'.decode()